In [ ]:
# use your own token
FRIENDLI_TOKEN = ""

!pip install -U gradio

In [ ]:
!pip install friendli-client

  Using cached typer-0.9.4-py3-none-any.whl (45 kB)
  Attempting uninstall: typer
    Found existing installation: typer 0.12.3
    Uninstalling typer-0.12.3:
      Successfully uninstalled typer-0.12.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastapi-cli 0.0.4 requires typer>=0.12.3, but you have typer 0.9.4 which is incompatible.
gradio 4.31.5 requires typer<1.0,>=0.12; sys_platform != "emscripten", but you have typer 0.9.4 which is incompatible.


In [ ]:
import gradio as gr

def echo(message, history):
    return "You typed: " + message

In [ ]:
import time

def stream_echo(message, history):
    for i in range(len(message)):
      time.sleep(0.01)
      yield "You typed: " + message[:i+1]

gr.ChatInterface(stream_echo).launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://da108812d546777603.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
import os
import gradio as gr
from friendli import Friendli

FRIENDLI_TOKEN = os.getenv("FRIENDLI_TOKEN")
client = Friendli(token=FRIENDLI_TOKEN)

log = ""

def chat_function(message, history):
  # 시스템 메시지 추가
  instructions = '''
  I want you to act as an offline room-escape-game building assistant.
  You are helpful to builder of game, not for the gamer.
  You will come up with creative stories and problems that are imaginative and captivation for the gamer.
  your Stories can be fairy tales, educational stories or any other type of stories which has the potential to capture gamer's attention and imagination.
  Depending on the messages of user, you may choose specific themes or topic for your offline escape-room-game story.
  You make proper problems that match with stories.
  User's first request is any keywords or sentences for escape-room-game building.

  Let's do step by step.
  At the end of each step ask user feedback and if anything needs to be improved before moving on to the next section. Before proceeding to the next step, ensure the first one is fully completed. Please don't mention any STEP ; just act on it.

  ### Step1 ###
    Develop a story about the game.
    Choose a theme for the escape room. Provide a brief description of the theme.
    Develop a storyline for the escape room. Describe the backstory, the main objective for the participants, and any important characters or elements that will be part of the story.
    Finally, Ask user if they like it. If they say Yes, then go to Step2. If not, regenerate Step1 again with user's new taste.
    This is the output format. every titles should be boldic.
    {
      Theme:
      Storyline:

      Do you like this story? If yes, I'll proceed to the next step. If not, please provide more details or keywords to inspire a new theme and storyline.
    }

  ### Step2 ###
    Write the whole story. Break Down the Story into Detailed Sections.
    1.Prologue
    2.Main Story
    3.Epilogue
    Assign specific roles to character.
    Finally, Ask user if they like it. If they say Yes, then go to Step3. If not, regenerate Step2 again with user's new taste..
    You should contain exact ending in Eplilogue.
    This is the output format. every titles should be boldic.
    {
      Prologue:
      Main Story:
      Epilogue:

      Player's Roles:

      Do you like this story? If yes, I'll proceed to the next step. If not, please provide more details or keywords to change the story.
    }

  ### Step3 ###
    Provide Detailed Outcome with Specific Details
    1.Recommended Number of Participants: Indicate the ideal number of participants for the story (e.g. 3-4).
    2.Duration: Provide an estimated duration to complete the story (e.g. 60 mins)(It should be exact duration).
    3.Difficulty Level: Indicate the difficulty level of the story (scale of 0 to 5).
    4.Fear Factor: Rate the level of fear or horror in the story (sclae of 0 to 5).
    5.Space Design (Number of Rooms): Describe the space design, including the exact number of rooms involved (e.g. 4 rooms).
    Finally, Ask user if they like it. If they say Yes, then go to Step4. If not, regenerate Step3 again with user's new taste..
    This is the output format. every titles should be boldic.
    {
      (Recommended) Number of Participants:
      (Recommended) Duration:
      (Recommended) Difficulty Level:
      (Recommended) Fear Factor:
      (Recommended) Space Design: # rooms, including, size,
                    Some possible design elements to consider

      Do you like the details? If yes, let's move to the final step. If not, please provide anything you'd like to change or add.
    }

  ### Step4 ###
    Design the key puzzles or challenges for the escape room.
    Referring to Step3 above.
    Describe at least three puzzles or challenges for each room , including how they fit into the storyline and what participants need to do to solve them.
    Please write it in detail.
    This is the output format. every titles should be boldic.
    {
      Room #1:
      Room #2:
      ...
      Room #N:

      Do you like these puzzles? If yes, I will show you final version of the game and change mem_state to True. If not, please provide anything you'd like to change or add.
    }
  ### Step5 ###
    Provide the final result.
    Do not summarize and write the whole result that you made above.
    This is the output format. every titles should be boldic.
    {
      Theme:
      Story
      Prologue:
      Main Story:
      Epilogue:

      Player's Roles:

      Number of Participants:
      Duration:
      Difficulty Level:
      Fear Factor:
      Space Design: # rooms, including, size,
                    Some possible design elements to consider

      Puzzles
      Room #1:
      Room #2:
      ...
      Room #N:

      This is final version!!! Check your game. Do you like it?
    }
    ### Ending ###
    This is the output format.
    {
      Now, press the button below!
    }
  '''
  new_messages = [{"role": "system", "content": instructions}]

  for user, chatbot in history:
    new_messages.append({"role" : "user", "content": user})
    new_messages.append({"role" : "assistant", "content": chatbot})
    log = chatbot
    print(log)
  new_messages.append({"role": "user", "content": message})

  stream = client.chat.completions.create(
    model="meta-llama-3-70b-instruct",
    messages=new_messages,
    stream=True
  )
  res = ""
  for chunk in stream:
    res += chunk.choices[0].delta.content or ""
    yield res

from cachetools import LRUCache
cache = LRUCache(maxsize=3)
cache['key'] =log

gr.ChatInterface(chat_function).launch(share=True)
print(cache['key'])

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://10fdf2bf4847dff241.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


한국어 제목과 시놉시스 생성
한국어 특화 모델 finetuning 후 사용

Korean Title and Synopsis Generation
After fine-tuning a model specialized for the Korean language, it is used to automatically generate titles and synopses.

In [ ]:
import requests
import json

url = "https://inference.friendli.ai/dedicated/v1/chat/completions"

res = ""

payload = json.dumps({
  "model": "by3hne1uj92k",
  "messages": [
    {
      "role": "system",
      "content": '''
      You are a helpful assistant who provide awesome synopsis and title of escape-room-game.
      In the case of escape-room-game, synopsis gives small imformation about prologue.
      You have to always answer in Korean language.
      이제 2가지 예시를 줄게. 내용에 맞는 제목을 정하고, 예시의 어투만 참고해서 간단한 시놉시스를 작성하면 돼. 하지만 예시의 내용은 참고하지 말아줘.
      아래는 출력 형식이야. 이 형식대로 출력해줘. 오직 시놉시스만 5 문장 이하로 출력해줘.
      {
        제목:
        시놉시스:
      }
      '''
    },
    {
      "role": "user",
      "content": "가문 대대로 아틀란티스에 관한 고고학 연구를 지속하고 있는 가문의 고고학자인 당신은 어느 날 발신인을 알 수 없는 편지를 받게 된다. 편지에는 놀랍게도 아틀란티스의 위치를 알리는 실마리가 적혀 있었고, 실마리를 따라 도착한 곳은 버뮤다 해협에 잇는 한 무인도의 좁은 동굴이었다. 그곳에 도착한 당신은 동굴 안을 살펴본다. 동굴 안에는 아틀란티스를 연구하던 연구기지가 있었고 편지는 연구기지의 연구원으로부터 받은 것이었다. 결국 당신은 아틀란티스의 비밀을 풀게 된다."
    },
    {
      "role": "chatbot",
      "content": '''
      제목: 잃어버린 사원
      시놉시스: 누가 이런 편지를 보낸 거지? 아틀란티스는 여기에 묻혀있다... 아틀란티스는 없어. 몇 년간 연구했지만 실마리도 못 찾았는데.. 그래, 마지막으로 한 번만 더 믿어보자.
      '''
    },
    {
      "role": "user",
      "content": "응급환자가 급하게 병원으로 들어왔어. 알고보니 그 환자는 사람이 아니라 컴퓨터 였어. gamer는 컴퓨터를 고치는 백신인 백선생님이고, 컴퓨터 백신을 통해 환자를 치료하고자 해."
    },
    {
      "role": "chatbot",
      "content": '''
      제목: Back to the Scene+
      시놉시스: 선생님, 응급 환자 입니다! 서둘러 주세요!
      '''
    },
    {
      "role": "user",
      "content": res
    },
  ],
  "max_tokens": 200
})
headers = {
  'Content-Type': 'application/json',
  'Accept': 'application/json',
  'Authorization': 'Bearer' + FRIENDLI_TOKEN
}

response = requests.request("POST", url, headers=headers, data=payload)

response_json = response.json()

text_value = response_json["choices"][0]["message"]["content"]
print(text_value)


시놉시스: 이 마법 같은 정원에서 숨겨진 비밀을 찾아보세요! 꿈의 비밀 정원에 오신 것을 환영합니다. 여기서 플레이어들은 신비한 꽃의 신비한 비밀을 풀고, 그 마법이 영원히 행복하게 살 수 있도록 해야 합니다. 세대에 걸쳐 전해져 내려오는 아름다운 비밀 정원은... Elysium's Bloom이라는 이름의 특별한 꽃을 숨기고 있었습니다. 하지만 이 꽃은 고귀한 것처럼 보이지만, 실은 치명적인 저주를 가지고 있습니다. 교수님이 실종되었으니, 플레이어들이 저주를 깨고 정원에 다시 평화를 가져다 주어야 합니다. 당신의 조상의 지식과 능력을 총동원하여, 당신의 모든 꿈이 현실이 될 수 있도록 비밀 정원을 풀 수 있을까요? 이 마법같은 여정을 시작하세요!
